In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from IPython.display import Image, display
#create root imAGE DIR
root_dir_images = '/kaggle/input/covid-chest-xray/images/'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img 

for i,imageName in enumerate(listOfImageNames,1):
    imgi= img.imread(imageName)
    fig.add_subplot(columns,rows,i)
    #display(Image(filename=imageName))
    plt.imshow(imgi)
    plt.show()

In [ ]:
pip install python-resize-image

from PIL import Image
from resizeimage import resizeimage
with open('/kaggle/input/covid-chest-xray/images/kjr-21-e24-g004-l-a.jpg','rb') as f:
    with Image.open(f) as image:
        cover = resizeimage.resize_thumbnail(image, [200, 200])
        #cover.save('test-image-cover.jpeg', image.format)
        display(cover)
        print(plt.imread(image))

to print image woth address first convert into  image them display
#display(Image('/kaggle/working/test-image-cover.jpeg'))

import os
from PIL import Image
from resizeimage import resizeimage
#loop over images address stored in the list and resize and display
for imageName in os.listdir(root_dir_images):
    #print(imageName)
    with open(os.path.join(root_dir_images,imageName),'rb') as f:
        with Image.open(f) as image:
            cover = resizeimage.resize_thumbnail(image, [224, 224,3])
            #display(cover)

In [ ]:
import os
import cv2 
scans=[]
i=0
#loop over images address stored in the list and resize and display
for imageName in os.listdir(root_dir_images):
    i+=1
    image=cv2.imread(os.path.join(root_dir_images,imageName))
    cover = cv2.resize(image, (224, 224))
    #plt.show(cover)
    scans.append(cover)

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/covid-chest-xray/metadata.csv')

In [ ]:
df.head()

In [ ]:
df=df[['Patientid','offset','sex','age','finding', 'survival','view','modality']]

In [ ]:
len(scans)==df.shape[0]
print(len(scans))
print(df.shape[0])

In [ ]:
    image=cv2.imread(os.path.join(root_dir_images,imageName))
    cover = cv2.resize(image, (224, 224))
    plt.imshow(cover)

In [ ]:
fig=plt.figure(figsize=(6,6))
columns = 2
rows = 5
for j,scan in enumerate(scans[0:10],1):
    fig.add_subplot(columns,rows,j)
    plt.imshow(scan)
plt.show()

In [ ]:
#df=df.loc[df['view']=='PA']
#df.shape #(92,8)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

In [ ]:
#label binarizer converts the labels into one hot vector directly
lb = LabelBinarizer()
#one hot encoded data labels. There are 6 findings including: covid19,sars,ards etc.
y=lb.fit_transform(df['finding'])
#y=to_categorical(y)
print(y[0])

In [ ]:
y=np.array(y)
scans=np.array(scans)/255

In [ ]:
pip install imutils

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import numpy as np
import argparse

In [ ]:
basemodel=VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
head=basemodel.output 
#output 2d shape2*2
head=AveragePooling2D(pool_size=(3,3))(head)
head=Flatten(name="flatten")(head)
head=Dense(64,activation="relu")(head)
head=Dropout(0.5)(head)
head=Dense(6,activation='softmax')(head)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model=Model(inputs=basemodel.input,outputs=head)

In [ ]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in basemodel.layers:
	layer.trainable = False

In [ ]:
# compile our model
epochs=10
lr=0.1
print("[INFO] compiling model...")
opt=Adam(lr=lr,decay=lr/epochs)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=["accuracy"])

In [ ]:
trainX,testX,trainY,testY=train_test_split(scans[0:146],y)
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest")


In [ ]:
BS=8
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=epochs)
